In [2]:
import math
import datetime
import multiprocessing as mp
import tushare as ts
import os
import numpy as np
import pandas as pd
import datetime
import arrow
p = print
# tushare 的初始化
token = '1212fb28fdc360c768e0bd5fca52afa9c8136809afad540f8cf11bc8'
pro = ts.pro_api(token)
# 获取当前上市公司列表，包括股票代码，注册地，行业，上市时间等数据
basic = pro.stock_basic(list_status = 'L')
# 这个 cell 是必须要载入的


# 

## 创业板的分析
20200824 首批创业板注册制首批企业正式挂牌上市.引发热度.
有很多20% 到 15% 的股票, 能不能发现一波...

首先要获得所有创业板股票的列表.


In [61]:
# 通过市场分,把股票分为: '中小板', '主板', '创业板', '科创板'
market_list = set(basic['market'])
p(market_list)
# grouth_list 创业板股票列表.
grouth_list = basic[basic['market'] == '创业板']

# 研究从 20200101 开始 : 20200824 日首批注册制正式挂牌.
study_starttime = '20200101'
# 平均线选择的list,目前选择 5, 10 ,20
averagechose_list = [5, 10 , 20, 30, 60]
###############################################
# 将两个数据分开存储:
study_criter = 10
# 有一天大于某个研究标准(目前是13%, 为了区别于过去的增长,所以设的稍微高一点)的股票存储在up10_dic中
up10_dic = dict()
# 其他储存在 no10_dic.
no10_dic = dict()
###############################################
#  iwant_len 标记我要取得的股票的数量,
iwant_len = 10
#################################################
# 取得股票数据:
for tscode_onestock in grouth_list['ts_code'] :
    #print('i start to study :', tscode_onestock)
    oneselect_stockdata = ts.pro_bar(api = pro, ts_code = tscode_onestock, start_date = study_starttime, adj = 'qfq', ma=averagechose_list)
    ############################################################
    # 目前把研究集中在8月份. 只在 所有数据的 前 25 个交易日中寻找符合标准的数据(8月内)
    analysisdata = oneselect_stockdata.iloc[:10]

    #if len(analysisdata[analysisdata['pct_chg'] > study_criter]) > 0:
    # 再不 简单粗暴,直接用 pct_chg 的和作为判定标准得了... 大于 20?...
    if sum(analysisdata['pct_chg']) > 20 :
        up10_dic[tscode_onestock] = oneselect_stockdata
        print('i get one stock:', tscode_onestock)
        #if len(up10_dic.keys()) > iwant_len :
         #   break
    else :
        no10_dic[tscode_onestock] = oneselect_stockdata
    




{'中小板', '主板', '科创板', '创业板'}
i get one stock: 300004.SZ
i get one stock: 300005.SZ
i get one stock: 300008.SZ
i get one stock: 300032.SZ
i get one stock: 300064.SZ
i get one stock: 300071.SZ
i get one stock: 300095.SZ
i get one stock: 300100.SZ
i get one stock: 300103.SZ
i get one stock: 300106.SZ
i get one stock: 300108.SZ
i get one stock: 300111.SZ
i get one stock: 300116.SZ
i get one stock: 300129.SZ
i get one stock: 300149.SZ
i get one stock: 300152.SZ
i get one stock: 300153.SZ
i get one stock: 300157.SZ
i get one stock: 300160.SZ
i get one stock: 300172.SZ
i get one stock: 300176.SZ
i get one stock: 300178.SZ
i get one stock: 300185.SZ
i get one stock: 300187.SZ
i get one stock: 300189.SZ
i get one stock: 300210.SZ
i get one stock: 300222.SZ
i get one stock: 300230.SZ
i get one stock: 300276.SZ
i get one stock: 300293.SZ
i get one stock: 300301.SZ
i get one stock: 300304.SZ
i get one stock: 300313.SZ
i get one stock: 300317.SZ
i get one stock: 300332.SZ
i get one stock: 300337.SZ


In [62]:
# 把上面的数据暂时储存起来.

import pickle
output = open('C:/Users/diaoh/Google 云端硬盘/量化面试资料/量化临时/quant for git/about-quant-/about-quant-/up10_dic_growthdata.pkl', 'wb')
pickle.dump(up10_dic, output)
output.close()

output_no10_dic = open('C:/Users/diaoh/Google 云端硬盘/量化面试资料/量化临时/quant for git/about-quant-/about-quant-/no10_dic_growthdata.pkl', 'wb')
pickle.dump(no10_dic, output_no10_dic)
output.close()

In [45]:
p(len(up10_dic.keys()))
p(len(no10_dic.keys()))

241
608


In [60]:
up10_dic.keys()

dict_keys(['300004.SZ', '300005.SZ', '300008.SZ', '300032.SZ', '300064.SZ', '300071.SZ', '300095.SZ', '300100.SZ', '300103.SZ', '300106.SZ', '300108.SZ'])

In [94]:
# 从 no10_dic 中挑出想要的股票, 
# 储存被选择的股票的 dict
select_dict = dict()
#####################
for one_key,one_value in no10_dic.items():
    #p(one_value)
    one_analysisdata = one_value.iloc[0]
    # 引入一个新的判定标准: k线不要被 
    # 首先:要最高价高于5日均线:
    if one_analysisdata['open'] > one_analysisdata['ma5'] or one_analysisdata['close'] > one_analysisdata['ma5'] : 

        # 其次: 5日 10 日均线非常接近?
        # 从 300100.SZ 在 20200824 的买入信号中, 
        #onetestdata = up10_dic['300100.SZ']
        #select_test = onetestdata[onetestdata['trade_date'] == '20200824'].iloc[0]
        #abs(select_test['ma10'] - select_test['ma5'])/select_test['ma10']
        # 从中暂时把这个设定为 0.01 : 及 ma5 和 ma10 在 1% 的范围内...
        ma5_from_ma10 = abs(one_analysisdata['ma10'] - one_analysisdata['ma5'])/one_analysisdata['ma10']
        #p(ma5_from_ma10)
        if ma5_from_ma10 > 0.01 :
            # 要是一个断小的k线:
            open_from_close = abs(one_analysisdata['open'] - one_analysisdata['close']) / one_analysisdata['close']
            if open_from_close < 0.025 :
                # 最低价 最好不要超过 时间最久的 均线?
                if one_analysisdata['low'] < one_analysisdata['ma20']:
                    select_dict[one_key] = one_value

#p(select_dict)
p(len(select_dict))


74


In [95]:
select_dict.keys()

dict_keys(['300002.SZ', '300006.SZ', '300054.SZ', '300081.SZ', '300088.SZ', '300093.SZ', '300098.SZ', '300109.SZ', '300121.SZ', '300142.SZ', '300151.SZ', '300162.SZ', '300173.SZ', '300177.SZ', '300198.SZ', '300219.SZ', '300236.SZ', '300238.SZ', '300275.SZ', '300289.SZ', '300294.SZ', '300302.SZ', '300315.SZ', '300324.SZ', '300326.SZ', '300342.SZ', '300379.SZ', '300389.SZ', '300394.SZ', '300396.SZ', '300406.SZ', '300426.SZ', '300438.SZ', '300457.SZ', '300463.SZ', '300474.SZ', '300479.SZ', '300512.SZ', '300519.SZ', '300523.SZ', '300525.SZ', '300558.SZ', '300559.SZ', '300567.SZ', '300573.SZ', '300577.SZ', '300579.SZ', '300584.SZ', '300599.SZ', '300603.SZ', '300615.SZ', '300618.SZ', '300627.SZ', '300628.SZ', '300633.SZ', '300634.SZ', '300638.SZ', '300672.SZ', '300674.SZ', '300675.SZ', '300741.SZ', '300746.SZ', '300758.SZ', '300768.SZ', '300770.SZ', '300818.SZ', '300823.SZ', '300825.SZ', '300840.SZ', '300841.SZ', '300842.SZ', '300848.SZ', '300855.SZ', '300858.SZ'])

In [87]:
onetestdata = up10_dic['300100.SZ']
one_analysisdata = onetestdata[onetestdata['trade_date'] == '20200824'].iloc[0]
abs(one_analysisdata['open'] - one_analysisdata['close']) / one_analysisdata['close']


0.02291325695581024

In [99]:
# 每一个十字星背后都是一波反弹么?
# 例子为 300559
# 研究从 20200101 开始:
# 
# 两个限定条件:
# 这是一个十字星
# 前面至少有一天是大跌:
# 超跌的结果,使得股价,可以看成最低价原理 5日均线.
# 对于 300559, 而言, 在20200807 , 20200619, 20200401, 20200317, 20200203,
# 它们的共同点是: 超跌,使得最低价严重远离 5日 均线..所以这个远离的距离是否可以当成一个判断反弹的标准呢?


#########################
# 研究从 20200101 开始 : 20200824 日首批注册制正式挂牌.
study_starttime = '20200101'
# 平均线选择的list,目前选择 5, 10 ,20
averagechose_list = [5, 10 , 20, 30, 60]



#################################################
# 取得股票数据:
#for tscode_onestock in grouth_list['ts_code'] :
tscode_onestock = '300559.SZ'
if tscode_onestock == '300559.SZ' :
    oneselect_stockdata = ts.pro_bar(api = pro, ts_code = tscode_onestock, start_date = study_starttime, adj = 'qfq', ma=averagechose_list)
    p(oneselect_stockdata)
    #break

ts_code trade_date     open     high      low    close  pre_close  \
0    300559.SZ   20200901  21.4900  21.5200  21.0800  21.2300    21.5300   
1    300559.SZ   20200831  21.8000  21.9500  21.5300  21.5300    21.5300   
2    300559.SZ   20200828  21.7800  21.7800  21.0500  21.5300    21.7700   
3    300559.SZ   20200827  21.4000  21.7700  21.1100  21.7700    21.3000   
4    300559.SZ   20200826  22.3200  22.4600  21.0600  21.3000    22.2300   
..         ...        ...      ...      ...      ...      ...        ...   
157  300559.SZ   20200108  20.1879  22.2893  19.9764  22.2893    20.2606   
158  300559.SZ   20200107  19.7253  20.4919  19.5337  20.2606    19.8178   
159  300559.SZ   20200106  19.1636  19.8178  18.9852  19.8178    19.2297   
160  300559.SZ   20200103  19.4346  19.5403  18.9456  19.2297    19.4279   
161  300559.SZ   20200102  19.5403  19.6196  18.9059  19.4279    19.3619   

     change  pct_chg        vol  ...      ma5     ma_v_5    ma10    ma_v_10  \
0   -0.3000  -1

In [128]:
# 反弹找到的list
bounce_list = ['20200728' , '20200619', '20200401', '20200317', '20200203']
# 最低点 到 5日均线的距离. list
low_from_ma5_list = []
# 寻找 ma5 和 ma10 远离距离的标准:
ma5_frafrom_ma10_list = []
###########################
for onedate in  bounce_list :
    one_bounce = oneselect_stockdata[oneselect_stockdata['trade_date'] == onedate].iloc[0]
    # 计算 最低点 到 5日均线的距离.
    low_from_ma5 = (one_bounce['low'] - one_bounce['ma5'])/ one_bounce['ma5']
    low_from_ma5_list.append(round(low_from_ma5, 4))
    ma5_frafrom_ma10 = (one_bounce['ma5'] - one_bounce['ma10']) / one_bounce['ma10']
    ma5_frafrom_ma10_list.append(round(ma5_frafrom_ma10, 4))
p(low_from_ma5_list)
p(ma5_frafrom_ma10_list)

[-0.0566, -0.034, -0.0388, -0.0718, -0.1127]
[-0.0274, -0.018, -0.0198, -0.0416, -0.0607]


In [130]:
# 将符合标准的日期储存起来
select_date = []
######################################
# 标准1 :
# 从 low_from_ma5_list 中,来判断反弹的条件.
# 如果去其中的最大值的话: 即 -0.034 即 -0.03 来作为反弹的条件.
# 判断如果距离 超过了 这个标准 反弹的概率.
low_f_ma5_crie = round(max(low_from_ma5_list), 2)
#################################
# 标准2 :
# 其次一个判断标准是: 
# 5日均线在 10日均线的下面, 并且远离.
# 还是先从 ma5_frafrom_ma10_list 中的最大值 作为标准进行判断.
ma5_f_ma10_crie = round(max(ma5_frafrom_ma10_list), 2)
######################################

###########
for i in range(len(oneselect_stockdata)):
    onedata = oneselect_stockdata.iloc[-i]
    low_from_ma5 = (onedata['low'] - onedata['ma5'])/ onedata['ma5']
    # 标准1 :
    if low_from_ma5 < low_f_ma5_crie :
        ma5_frafrom_ma10 = (onedata['ma5'] - onedata['ma10']) / onedata['ma10']
        # 标准2 :
        if ma5_frafrom_ma10 < ma5_f_ma10_crie:
            select_date.append(onedata['trade_date'])


            break

p(select_date)
p(bounce_crie)
p(ma5_f_ma10_crie)

['20200121']
-0.03
-0.02


In [117]:
oneselect_stockdata

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,...,ma5,ma_v_5,ma10,ma_v_10,ma20,ma_v_20,ma30,ma_v_30,ma60,ma_v_60
0,300559.SZ,20200901,21.4900,21.5200,21.0800,21.2300,21.5300,-0.3000,-1.3934,36698.48,...,21.4720,46169.134,21.938,52919.324,22.0910,71640.2535,21.9197,69906.6460,22.0472,75908.9405
1,300559.SZ,20200831,21.8000,21.9500,21.5300,21.5300,21.5300,0.0000,0.0000,39360.33,...,21.6720,50470.926,22.136,64161.762,22.1075,72707.9550,21.9920,71730.2337,22.0627,75694.6005
2,300559.SZ,20200828,21.7800,21.7800,21.0500,21.5300,21.7700,-0.2400,-1.1024,50236.11,...,21.9060,52418.428,22.349,72932.953,22.1110,73589.2175,22.0360,72108.0180,22.0697,75380.4182
3,300559.SZ,20200827,21.4000,21.7700,21.1100,21.7700,21.3000,0.4700,2.2066,34787.86,...,22.1160,54799.608,22.546,76693.985,22.1010,74568.2900,22.0733,72677.4330,22.0790,75130.7508
4,300559.SZ,20200826,22.3200,22.4600,21.0600,21.3000,22.2300,-0.9300,-4.1835,69762.89,...,22.1840,58268.928,22.675,85361.792,22.0575,75746.2965,22.0943,74220.0643,22.0883,75222.8117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,300559.SZ,20200108,20.1879,22.2893,19.9764,22.2893,20.2606,2.0287,10.0130,121643.34,...,20.2051,60784.048,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
158,300559.SZ,20200107,19.7253,20.4919,19.5337,20.2606,19.8178,0.4428,2.2344,56365.40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
159,300559.SZ,20200106,19.1636,19.8178,18.9852,19.8178,19.2297,0.5881,3.0583,42473.64,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
160,300559.SZ,20200103,19.4346,19.5403,18.9456,19.2297,19.4279,-0.1982,-1.0202,37254.59,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
